## Deep learning using Keras

In [5]:
import tensorflow.keras as keras
from sklearn.datasets import fetch_openml
import numpy as np

In [2]:
X, Y = fetch_openml('mnist_784', version=1, return_X_y=True) 

In [3]:
print ('data shape: ', X.shape)
print ('label shape:', Y.shape)

data shape:  (70000, 784)
label shape: (70000,)


In [6]:
# Set the randomizer seed so results are the same each time.
np.random.seed(0)

# Rescale grayscale values to [0,1]. This is the only pre processing we'll do on this dataset!
X = X / 255.0

# Shuffle the input.
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]


# Set some variables to hold test, dev, and training data.
test_data, test_labels = X[61000:], Y[61000:]
dev_data, dev_labels = X[60000:61000], Y[60000:61000]
train_data, train_labels = X[:60000], Y[:60000]

# Creating a mini training set to get quick initial analysis for computationally heavy algorithms
mini_train_data, mini_train_labels = X[:1000], Y[:1000]

### Simple 2 hidden layer fully connected network

In [101]:
inputs = keras.layers.Input(shape = (784,))
h1 = keras.layers.Dense(128, activation='relu')(inputs)
h1 = keras.layers.Dropout(.25)(h1)
h2 = keras.layers.Dense(64, activation='relu')(h1)
outputs = keras.layers.Dense(10, activation='softmax')(h2)

model1 = keras.models.Model(inputs = inputs, outputs = outputs)
model1.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [53]:
categorial_train_labels = keras.utils.to_categorical(train_labels)

In [102]:
model1.fit(train_data, categorial_train_labels, epochs = 2, batch_size=100)

Epoch 1/2
60000/60000 [==============================] - 2s 37us/sample - loss: 0.3519 - acc: 0.8968
Epoch 2/2
60000/60000 [==============================] - 2s 33us/sample - loss: 0.1658 - acc: 0.9506


In [103]:
def result(model, test_data, test_labels):
    pred = model.predict(test_data)
    pred = np.argmax(pred, axis = 1)
    return sum(pred == list(map(int, test_labels)))/test_labels.shape[0]

In [104]:
result(model1, test_data, test_labels)

0.9651111111111111

In [93]:
model1.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 784)]             0         
_________________________________________________________________
dense_43 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_44 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_45 (Dense)             (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


### 2 CNN layers followed by a fully connected hidden layer

In [153]:
cinputs = keras.layers.Input(shape = (28,28,1))
c1 = keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(cinputs)
#c1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(c1)
c2 = keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu')(c1)
c3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(c2)
c3 = keras.layers.Dropout(.25)(c3)
c4 = keras.layers.Flatten()(c3)
c5 = keras.layers.Dense(128, activation='relu')(c4)
c5 = keras.layers.Dropout(.25)(c5)
coutputs = keras.layers.Dense(10, activation='softmax')(c5)

model2 = keras.models.Model(inputs = cinputs, outputs = coutputs)
model2.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [115]:
model2_train_data = train_data.reshape(-1, 28, 28, 1)
model2_test_data = test_data.reshape(-1, 28, 28, 1)

In [154]:
model2.fit(model2_train_data, categorial_train_labels, epochs = 3, batch_size=1000)

Epoch 1/3
60000/60000 [==============================] - 77s 1ms/sample - loss: 0.4922 - acc: 0.8469
Epoch 2/3
60000/60000 [==============================] - 79s 1ms/sample - loss: 0.1207 - acc: 0.9635
Epoch 3/3
60000/60000 [==============================] - 82s 1ms/sample - loss: 0.0749 - acc: 0.9768


In [155]:
result(model2, model2_test_data, test_labels)

0.9816666666666667

In [156]:
model2.summary()

Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_81 (Dense)             (None, 128)               117